In [1]:
!pip install psycopg2

In [2]:
import time

import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# Database connection parameters
db_host = "pg_container"
db_name = "test_db"
db_name2 = "example_db"
db_user = "shoc"
db_password = "password"

In [3]:
#conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
conn=psycopg2.connect( 
    database=db_name, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port="5432"
) 

#conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
conn.autocommit = True
cur = conn.cursor()

In [4]:
query = f"CREATE DATABASE {db_name2}"
cur.execute(query)
conn.close()

In [5]:
conn2=psycopg2.connect( 
    database=db_name2, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port="5432"
) 
cur = conn2.cursor()
conn2.close()

In [6]:
conn=psycopg2.connect( 
    database=db_name, 
    user=db_user, 
    password=db_password, 
    host=db_host, 
    port="5432"
) 

#conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
conn.autocommit = True
cur = conn.cursor()

query = f"DROP DATABASE {db_name2}"
cur.execute(query)
conn.close()

In [7]:
-- Table with CHAR
CREATE TABLE CharTable (
    ID SERIAL PRIMARY KEY,
    Name CHAR(50),
    OtherData TEXT -- Add other relevant columns
);

-- Table with VARCHAR
CREATE TABLE VarcharTable (
    ID SERIAL PRIMARY KEY,
    Name VARCHAR(50),
    OtherData TEXT -- Add other relevant columns
);

-- Insert data (ensure data is less than 50 chars for 'Name')
INSERT INTO CharTable (Name, OtherData)
SELECT md5(random()::text), 'Some additional text'
FROM generate_series(1, 100000); -- 100,000 rows

INSERT INTO VarcharTable (Name, OtherData)
SELECT Name, OtherData FROM CharTable; -- copy data from CharTable

-- Optionally, add an index on the Name column (after the data is loaded)
-- CREATE INDEX idx_char_name ON CharTable (Name);
-- CREATE INDEX idx_varchar_name ON VarcharTable (Name);

SyntaxError: invalid syntax (3136918032.py, line 1)

In [ ]:
-- Simple SELECT
SELECT * FROM CharTable;
SELECT * FROM VarcharTable;

-- SELECT with WHERE (handling trailing spaces for CHAR)
SELECT * FROM CharTable WHERE TRIM(Name) = 'your_test_string';
SELECT * FROM VarcharTable WHERE Name = 'your_test_string';

-- Aggregated query
SELECT COUNT(*) FROM CharTable WHERE OtherData LIKE '%text%';
SELECT COUNT(*) FROM VarcharTable WHERE OtherData LIKE '%text%';

-- ORDER BY
SELECT * FROM CharTable ORDER BY TRIM(Name);
SELECT * FROM VarcharTable ORDER BY Name;

In [ ]:
import psycopg2
import time

# Database connection parameters
db_host = "pg_container"
db_name = "test_db"
db_user = "shoc"
db_password = "password"

def execute_query(query):
    conn = None
    try:
        conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)
        cur = conn.cursor()

        start_time = time.time()
        cur.execute(query)
        rows = cur.fetchall()
        end_time = time.time()

        elapsed_time = end_time - start_time
        return elapsed_time

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn:
            cur.close()
            conn.close()

# Define your test queries
char_query = "SELECT * FROM CharTable WHERE TRIM(Name) = 'your_test_string';"
varchar_query = "SELECT * FROM VarcharTable WHERE Name = 'your_test_string';"

# Number of runs
num_runs = 10

# Run the tests and time them
char_times = []
varchar_times = []

for _ in range(num_runs):
    char_times.append(execute_query(char_query))
    varchar_times.append(execute_query(varchar_query))

# Calculate averages (excluding the first run)
avg_char_time = sum(char_times[1:]) / (num_runs - 1)
avg_varchar_time = sum(varchar_times[1:]) / (num_runs - 1)

print(f"Average CHAR query time: {avg_char_time:.6f} seconds")
print(f"Average VARCHAR query time: {avg_varchar_time:.6f} seconds")